In [3]:
import os
import sys
import glob

import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.backend.tensorflow_backend import set_session

import matplotlib.pyplot as plt

import nibabel as nib

Using TensorFlow backend.


In [4]:
def plot(img):
    plt.figure()
    plt.imshow(img[:, 40, :], cmap='gray')
    
def load_nii(path):
    nii = nib.load(path)
    vol = nii.get_data().astype(np.float32)
    return np.squeeze(vol)

In [5]:
run_dirs = glob.glob('../runs/*')
run_dirs = sorted(run_dirs)

print('found {} runs'.format(len(run_dirs)), '\n')

for i, run in enumerate(run_dirs):
    print(i, run, '\n')

found 9 runs 

0 ../runs/gan_20190415_1849_gpu=0_bs=4_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25_vr=1.0_ti=False_is=5_cs=5_rf=10_rs=5_glw=[100_10_100_1]_clw=[1_1_10]_tag= 

1 ../runs/gan_20190415_1857_gpu=1_bs=4_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=5_glw=[100_10_100_1]_clw=[1_1_10]_tag= 

2 ../runs/gan_20190415_1939_gpu=2_bs=4_enc=[16_32_64_128]_dec=[128_64_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=25_glw=[100_10_100_1]_clw=[1_1_10]_tag=brains_deeper_gen 

3 ../runs/gan_20190415_2231_gpu=0_bs=4_enc=[16_32_64_128]_dec=[128_64_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=500.0_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=25_glw=[100_10_100_1]_clw=[1_1_10]_tag=brains_deep_high_pl 

4 ../runs/gan_20190416_0011_gpu=1_bs=4_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=25_glw=[100_10_100_1]_clw=[1_1_10]_tag=brains_regular_gen 

5 ../runs/gan_201

In [6]:
# set run dir
run_dir = run_dirs[6]

pre_dirs = glob.glob(os.path.join(glob.escape(run_dir), 'test', '*'))
pre_dirs = sorted(pre_dirs)

print('found {} pre_dirs'.format(len(pre_dirs)))

for i, pre_dir in enumerate(pre_dirs):
    print(i, os.path.basename(pre_dir))

found 2 pre_dirs
0 gen_055
1 gen_120


In [5]:
# generator config, run_dir, etc...
pre_dir = pre_dirs[5]

csv_path = os.path.join(pre_dir, 'meta.csv')
csv = pd.read_csv(csv_path)
print(csv.shape)

(271, 38)


In [19]:
# classifier
run_dir = '../../../../backup/agemorph/models/clf/clf_20190330_1549_gpu=4_bs=8_lr=0.001_b1=0.9_b2=0.999_ep=0.1_bn=True_ds=1_lw=[1.0]/'
model_name = 'clf_100'
model_file = os.path.join(run_dir, model_name+'.h5')

if os.path.isfile(model_file):
    print('found model file')

found model file


In [20]:
# gpu config
gpu_id = 7

if gpu_id is not None:                                                     
    gpu = '/gpu:' + str(gpu_id)                                            
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)                       
    config = tf.ConfigProto()                                              
    config.gpu_options.allow_growth = True                                 
    config.allow_soft_placement = True                                     
    set_session(tf.Session(config=config))                                 
else:                                                                      
    gpu = '/cpu:0'

In [21]:
print('starting predict')

with tf.device(gpu):
    
    # load classifier model
    classifier = keras.models.load_model(model_file)
    
    for i, (_, row) in enumerate(csv.iterrows()):
        
        if i % 10 == 0:
            print('step', i)
        
        xr = load_nii(row['img_path_0'])[None, ..., None]
        yr = load_nii(row['img_path_1'])[None, ..., None]
        yf = load_nii(row['img_path_yf'])[None, ..., None]
        
        batch = np.concatenate([xr, yr, yf], axis=0)
        
        pred = classifier.predict([batch])
        
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_xr'] = pred[0, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yr'] = pred[1, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yf'] = pred[2, 1]
        
print('finished predict')

csv_out_path = os.path.join(pre_dir, model_name+'.csv')

print('writing csv to {}'.format(csv_out_path))

csv.to_csv(csv_out_path, index=False)

starting predict
step 0
step 10
step 20
step 30
step 40
step 50
step 60
step 70
step 80
step 90
step 100
step 110
step 120
step 130
step 140
step 150
step 160
step 170
step 180
step 190
step 200
step 210
step 220
step 230
step 240
step 250
step 260
step 270
finished predict
writing csv to ../runs/gan_20190416_0011_gpu=1_bs=4_enc=[16_32_32_32]_dec=[32_32_32_32_16_8]_cbn=16_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_vr=1.0_ti=True_is=5_cs=5_rf=10_rs=25_glw=[100_10_100_1]_clw=[1_1_10]_tag=brains_regular_gen/test/gen_450_10/clf_100.csv


In [22]:
clf_outs = glob.glob(os.path.join(glob.escape(pre_dir), '[!meta]*.csv'))
clf_outs = sorted(clf_outs)

for i, clf_out in enumerate(clf_outs):
    print(i, os.path.basename(clf_out))

0 clf_100.csv
1 clf_500.csv


In [23]:
csv_path = clf_outs[0]
csv = pd.read_csv(csv_path)
print(csv.shape)

(271, 42)


In [11]:
csv = csv.sample(frac=1).reset_index(drop=True)

pMCI = csv[csv.pat_dx_1 == 3].img_id_0.values
sMCI = csv[csv.pat_dx_1 == 2].img_id_0.values

p_splits = np.split(pMCI, np.linspace(0, len(pMCI), 6).astype(int)[1:-1])
s_splits = np.split(sMCI, np.linspace(0, len(sMCI), 6).astype(int)[1:-1])

In [24]:
for i, (p_split, s_split) in enumerate(zip(p_splits, s_splits)):
    csv.loc[csv.img_id_0.isin(p_split), 'cv_split'] = i
    csv.loc[csv.img_id_0.isin(s_split), 'cv_split'] = i

In [25]:
def acc_threshold(csv, col, t):
    true_pos = ((csv[col] > t) & (csv.pat_dx_1 == 3)).sum()
    true_neg = ((csv[col] <= t) & (csv.pat_dx_1 == 2)).sum()
    false_pos = ((csv[col] > t) & (csv.pat_dx_1 == 2)).sum()
    false_neg = ((csv[col] <= t) & (csv.pat_dx_1 == 3)).sum()

    acc_p = true_pos / (true_pos + false_neg)
    acc_s = true_neg / (true_neg + false_pos)

    balanced_acc = (acc_p + acc_s) / 2

    true = true_pos + true_neg
    false = false_pos + false_neg
    
    return (round(balanced_acc, 3), true, false, round(acc_p, 2), round(acc_s, 2), round(t, 2))

In [26]:
def best_threshold(csv, col):
    l = []
    
    for t in np.arange(0, 1, 0.01):
        l.append(acc_threshold(csv, col, t))
        
    return max(l)

In [27]:
def acc_cv(csv_in, col, cv_split):
    
    csv = csv_in[csv_in.cv_split != cv_split]
    
    t = best_threshold(csv, col)[5]
    print(t)
    
    csv = csv_in[csv_in.cv_split == cv_split]
    
    acc = acc_threshold(csv, col, t)
    print(acc)
    
    return acc

In [28]:
l = [acc_cv(csv, 'pAD_yf', i) for i in range(5)]

np.array(list(zip(*l))[0]).mean()

0.02
(0.522, 30, 23, 0.37, 0.68, 0.02)
0.01
(0.675, 36, 19, 0.75, 0.6, 0.01)
0.08
(0.567, 33, 20, 0.37, 0.76, 0.08)
0.01
(0.675, 36, 19, 0.75, 0.6, 0.01)
0.01
(0.736, 41, 14, 0.7, 0.77, 0.01)


0.635

In [29]:
l = [acc_cv(csv, 'pAD_xr', i) for i in range(5)]

np.array(list(zip(*l))[0]).mean()

0.01
(0.601, 33, 20, 0.53, 0.68, 0.01)
0.01
(0.689, 37, 18, 0.75, 0.63, 0.01)
0.01
(0.713, 39, 14, 0.63, 0.79, 0.01)
0.01
(0.643, 36, 19, 0.6, 0.69, 0.01)
0.01
(0.657, 37, 18, 0.6, 0.71, 0.01)


0.6606

In [18]:
l = [acc_cv(csv, 'pAD_yr', i) for i in range(5)]

np.array(list(zip(*l))[0]).mean()

0.04
(0.633, 32, 21, 0.74, 0.53, 0.04)
0.01
(0.679, 34, 21, 0.9, 0.46, 0.01)
0.47
(0.71, 38, 15, 0.68, 0.74, 0.47)
0.01
(0.786, 40, 15, 1.0, 0.57, 0.01)
0.24
(0.736, 41, 14, 0.7, 0.77, 0.24)


0.7088000000000001